# Regressor AutoML - Experimento

Este é um componente que utiliza a biblioteca [auto-sklearn](https://github.com/automl/auto-sklearn) para obter um ou mais modelos regressores já otimizados. <br>
O auto-sklearn é um kit de ferramentas de machine learning automatizado e um substituto para [estimator](https://scikit-learn.org/stable/glossary.html#term-estimators) do [scikit-learn](https://scikit-learn.org/stable/).

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parameters
dataset = "/tmp/data/boston.csv" #@param {type:"string"}
target = "medv" #@param {type:"feature", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}

# selected features to perform the model
filter_type = "remover" #@param ["incluir","remover"] {type:"string",multiple:false,label:"Modo de seleção das features",description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção 'incluir'. Caso deseje informar as features que não devem ser utilizadas, selecione 'remover'. "}
model_features = "" #@param {type:"feature",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

# features to apply One Hot Encoder
one_hot_features = "" #@param {type:"feature", multiple:true, label:"Features para fazer codificação one-hot", description:"Seu modelo utilizará a codificação one-hot para as features selecionadas. As demais features categóricas serão codificadas utilizando a codificação ordinal."}

# hyperparameters
time_left_for_this_task = 60 #@param {type:"integer", label:"Tempo máximo de busca", description:"Limite de tempo (em segundos) para a procura de modelos apropriados."}
per_run_time_limit = 60 #@param {type:"integer", label:"Tempo máximo para ajuste de modelo", description:"Limite de tempo (em segundos), para uma única chamada, para ajuste de um modelo de Machine Learning."}
ensemble_size = 50 #@param {type:"integer", label:"Ensemble Learning", description:"Número de modelos adicionados ao conjunto criado pela seleção do Ensemble das bibliotecas de modelos."}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [ ]:
df.dropna(subset=[target], inplace=True)
y = df[target].to_numpy()

## Filtragem das features 

Seleciona apenas as features que foram declaradas no parâmetro model_features. Se nenhuma feature for especificada, todo o conjunto de dados será utilizado para a modelagem.

In [ ]:
columns_to_filter = columns

if len(model_features) >= 1:

    if filter_type == "incluir":
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        featuretypes = featuretypes[columns_index]
    else:
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns, columns_index)
        featuretypes = np.delete(featuretypes, columns_index)

# keep the features selected
df_model = df[columns_to_filter]
X = df_model.to_numpy()

## Divide dataset em subconjuntos de treino e teste

Subconjunto de treino: amostra de dados usada para treinar o modelo.<br>
Subconjunto de teste: amostra de dados usada para fornecer uma avaliação imparcial do treinamento do modelo no subconjunto de dados de treino.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

## Configuração das features

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]


# Selects non-numerical features to apply ordinal encoder or one-hot encoder
one_hot_features = np.asarray(one_hot_features)
non_numerical_indexes_one_hot = np.where(
    ~(featuretypes == NUMERICAL) & np.isin(columns_to_filter, one_hot_features)
)[0]
non_numerical_indexes_ordinal = np.where(
    ~(featuretypes == NUMERICAL) & ~(np.isin(columns_to_filter, one_hot_features))
)[0]


# After the step handle_missing_values,
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = np.arange(len(numerical_indexes))
non_numerical_indexes_after_handle_missing_values = np.arange(
    len(numerical_indexes), len(featuretypes)
)
one_hot_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_one_hot))[0]
]
ordinal_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_ordinal))[0]
]

## Treina um modelo usando autosklearn.regression.AutoSklearnRegressor

In [ ]:
from autosklearn.regression import AutoSklearnRegressor
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    steps=[
        (
            "handle missing values",
            ColumnTransformer(
                [
                    ("imputer_mean", SimpleImputer(strategy="mean"), numerical_indexes),
                    (
                        "imputer_mode",
                        SimpleImputer(strategy="most_frequent"),
                        non_numerical_indexes,
                    ),
                ],
                remainder="drop",
            ),
        ),
        (
            "handle categorical features",
            ColumnTransformer(
                [
                    (
                        "feature_encoder_ordinal",
                        OrdinalEncoder(),
                        ordinal_indexes_after_handle_missing_values,
                    ),
                    (
                        "feature_encoder_onehot",
                        OneHotEncoder(),
                        one_hot_indexes_after_handle_missing_values,
                    ),
                ],
                remainder="passthrough",
            ),
        ),
        (
            "estimator",
            AutoSklearnRegressor(
                time_left_for_this_task=time_left_for_this_task,
                per_run_time_limit=per_run_time_limit,
                ensemble_size=ensemble_size,
            ),
        ),
    ]
)

pipeline.fit(X_train, y_train)
X_train = Pipeline(pipeline.steps[:-1]).transform(X_train)
pipeline.named_steps.estimator.refit(X_train, y_train)

features_after_pipeline = np.concatenate(
    (columns_to_filter[numerical_indexes], columns_to_filter[non_numerical_indexes])
)

## Avaliação de desempenho

O [Coeficiente de determinação](https://pt.wikipedia.org/wiki/Coeficiente_de_determinação) (ou R²), corresponde à correlação ao quadrado entre os valores de resultado observados e os valores previstos pelo modelo.

In [ ]:
from sklearn.metrics import r2_score

# uses the model to make predictions on the Test Dataset
y_pred = pipeline.predict(X_test)

# computes R²
r2 = r2_score(y_test, y_pred)

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [ ]:
from platiagro import save_metrics

save_metrics(r2_score=r2)

## Visualiza resultados

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


def annotate_plot(e, s, plt, y_lim, h, abs_err):
    if h < 2:
        p = 0.05
    else:
        p = 0.1
    plt.annotate(
        "",
        xy=(max(e), y_lim[1] / h),
        xytext=(0, y_lim[1] / h),
        arrowprops=dict(arrowstyle="->"),
    )
    plt.annotate(
        "",
        xy=(min(e), y_lim[1] / h),
        xytext=(0, y_lim[1] / h),
        arrowprops=dict(arrowstyle="->"),
    )
    plt.annotate("{}%".format(s), xy=(0, (1 + p) * y_lim[1] / h), ha="center")
    if abs_err:
        plt.annotate(
            "{:.2f}".format(max(e)),
            xy=((0 + max(e)) / 2, (1 - p) * y_lim[1] / h),
            ha="center",
        )
        plt.annotate(
            "{:.2f}".format(min(e)),
            xy=((0 + min(e)) / 2, (1 - p) * y_lim[1] / h),
            ha="center",
        )
    else:
        plt.annotate(
            "{:.2f}%".format(100 * max(e)),
            xy=((0 + max(e)) / 2, (1 - p) * y_lim[1] / h),
            ha="center",
        )
        plt.annotate(
            "{:.2f}%".format(100 * min(e)),
            xy=((0 + min(e)) / 2, (1 - p) * y_lim[1] / h),
            ha="center",
        )

In [ ]:
abs_err = False
if any(y_test == 0):
    err = y_pred - y_test
    abs_err = True
else:
    err = (y_pred - y_test) / y_test

sorted_idx = np.argsort(np.abs(err))
n = int(0.7 * len(y_test))
idx = sorted_idx[:n]
e = err[idx]

n = int(0.95 * len(y_test))
idx = sorted_idx[:n]
aux = err[idx]
x_lim = (aux.min(), aux.max())

plt.figure()

kde = gaussian_kde(err)
x_err = np.linspace(err.min(), err.max(), 1000)
p_err = kde(x_err)
plt.plot(x_err, p_err, "b-")

y_lim = plt.ylim()
plt.ylim((0, y_lim[1]))
y_lim = plt.ylim()
plt.xlim(x_lim)
plt.plot([e.min(), e.min()], y_lim, "r--")
plt.plot([e.max(), e.max()], y_lim, "r--")

# Shade the area between e.min() and e.max()
plt.fill_betweenx(
    y_lim,
    e.min(),
    e.max(),
    facecolor="red",  # The fill color
    color="red",  # The outline color
    alpha=0.2,
)  # Transparency of the fill

annotate_plot(e, 70, plt, y_lim, 2, abs_err)
annotate_plot(aux, 95, plt, y_lim, 1.2, abs_err)

plt.grid(True)
plt.title("Error Distribution")

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
pipeline.fit(X, y)

y_pred = pipeline.predict(X)
new_columns = "AutoMLRegressor" + "_prediction"
df[new_columns] = y_pred

# save dataset changes
df.to_csv(dataset, index=False)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [ ]:
from joblib import dump

artifacts = {
    "pipeline": pipeline,
    "columns": columns,
    "columns_to_filter": columns_to_filter,
    "new_columns": new_columns,
    "features_after_pipeline": features_after_pipeline,
}

dump(artifacts, "/tmp/data/automl-regressor.joblib")